## **LangChain**



- Framework para desarrollar apps basadas en LLMs

### Instalación

In [ ]:
! pip install langchain
! pip install --upgrade --quiet  langchain
! pip install langchain-community
! pip install langchainhub
! pip install langchain-openai
! pip install chromadb bs4

In [41]:
from dotenv import load_dotenv
load_dotenv()
import os
key = "Hello"

### **Componentes : LLM**

- OpenAI: Chat Model

Inicia el modelo

In [42]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [43]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


In [44]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Write a recipe using {ingredient}.")

chain = prompt | llm

In [45]:
response = chain.invoke({"ingredient": "tomato"}).content

In [46]:
print(response)

Recipe: Tomato Basil Bruschetta

Ingredients:
- 4 ripe tomatoes, diced
- 1/4 cup fresh basil leaves, chopped
- 2 cloves garlic, minced
- 2 tablespoons extra virgin olive oil
- 1 tablespoon balsamic vinegar
- Salt and pepper, to taste
- Baguette or crusty bread, sliced
- Optional toppings: mozzarella cheese, prosciutto, or avocado

Instructions:
1. In a medium bowl, combine the diced tomatoes, chopped basil, minced garlic, olive oil, and balsamic vinegar. Mix well to combine.
2. Season the tomato mixture with salt and pepper according to your taste. Set aside and let it marinate for at least 15 minutes to allow the flavors to meld together.
3. Preheat your oven to 375°F (190°C). Arrange the sliced baguette or crusty bread on a baking sheet.
4. Brush each slice of bread with a little olive oil and place them in the preheated oven. Bake for about 8-10 minutes or until the bread turns golden and crispy.
5. Remove the bread from the oven and let it cool slightly. If desired, you can rub eac

- OpenAI: Embeddings Model

In [47]:
from langchain_openai import OpenAIEmbeddings

In [54]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [59]:
# Embed the docs or text (in this case it will be the tomato recipe)
text = response 
doc_result = embeddings.embed_documents([text])
doc_result[0][:5]

[-0.02423724047655008,
 -0.028520545865452353,
 -0.01385283698896719,
 0.03834080813674366,
 -0.017206350869055486]

In [58]:
# Embed the query
query = "How many ingredients will I need for this recipe?"
query_result = embeddings.embed_query(text)
query_result[:5]

[-0.02423724047655008,
 -0.028520545865452353,
 -0.01385283698896719,
 0.03834080813674366,
 -0.017206350869055486]

### **Componentes : Chains**


### **Q&A : Indexing**

- Cargar texto (documentos, base de conocimiento)
- Trocear el texto para poder hacer llamadas a la API
- Almacenar el contenido

In [60]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [62]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = "What is the agent-based model?"
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)